In [1]:
from modules_th.video_block import *
from modules_th.triplet_loss import *
from modules_th.inflator import *
from fastai.vision.all import *
from pathlib import Path
import torch.nn as nn
import torch
import time

KeyboardInterrupt: 

In [2]:
class MyResizeTime(Transform):
    split_idx = None # 0- train 1- validation 
    def __init__(self, skip=2, l=50, drop_last=True,**kwargs): 
        self.skip = skip
        self.l = l
        self.drop_last = drop_last
        super().__init__(**kwargs)
        
    def encodes(self, vid:Video, split_idx=split_idx):
        '''create a list of frame-images (snippet) out a single video path'''
        l, skip, l_vid = self.l, self.skip, len(vid)
        if l_vid > l:
            idx = (len(vid)-l)//2 if split_idx else random.randint(0,len(vid)-l) # ** if validation always takes middle snip
            return vid[idx:idx+l]        
        else:
            vid = stretch(vid, l)         
        return vid
 

In [ ]:
path_charades = Path('/mnt/data/adrianlopez/Videos/Charades') 

using_df0 = True

n = 0 if using_df0 else ''
items = pd.read_csv(path_charades/f'df{n}.csv', index_col=0).iloc[2271*12:]

In [6]:
get_paths = DF2Paths(path_charades, fps=24) if using_df0 else lambda df: df['paths']
get_lbls = lambda df: df['lbl']

vid_pl = [get_paths, Video.create, MyResizeTime(l=50), Resize(128), ToTensor(), IntToFloatTensor()]
lbl_pl = [get_lbls, Categorize()]
splits = RandomSplitter(seed=42)(items)

ds = Datasets(items, [vid_pl, lbl_pl], splits=splits)

In [7]:
bs = 12
t = time.perf_counter()
for i, o in enumerate(ds.train):
    if i >= bs: break
time.perf_counter() - t

7.259147342000006

In [8]:
n_el, n_lbl = 3, 4
bs = n_lbl * n_el
us = UniformizedShuffle(items['lbl'].iloc[splits[0]], n_el=n_el, n_lbl=n_lbl)

dls = ds.dataloaders(bs=bs, shuffle_fn=us, shuffle_train=False)

In [9]:
t0 = time.perf_counter()
tb0 = t0
for i, (b, _) in enumerate(dls.train):
    tb1 = time.perf_counter()
    T = tb1 - t0
    print(f'Finished Batches: {i+1}',
          f'\nTime for last batch: {tb1 - tb0:.2f}', 
          f'\nTotal time: {T:.2f}',
          f'\nAvg time per batch: {T/(i+1):.2f}\n')            
    tb0 = tb1

time.perf_counter() - t

Finished Batches: 0 
Time for last batch: 3.49 
Total time: 3.49 
Avg time per batch: 3.49

Finished Batches: 1 
Time for last batch: 108.88 
Total time: 112.37 
Avg time per batch: 56.19

Finished Batches: 2 
Time for last batch: 61.86 
Total time: 174.24 
Avg time per batch: 58.08

Finished Batches: 3 
Time for last batch: 33.90 
Total time: 208.14 
Avg time per batch: 52.03

Finished Batches: 4 
Time for last batch: 0.18 
Total time: 208.31 
Avg time per batch: 41.66

Finished Batches: 5 
Time for last batch: 0.19 
Total time: 208.51 
Avg time per batch: 34.75

Finished Batches: 6 
Time for last batch: 0.10 
Total time: 208.61 
Avg time per batch: 29.80

Finished Batches: 7 
Time for last batch: 29.09 
Total time: 237.70 
Avg time per batch: 29.71

Finished Batches: 8 
Time for last batch: 0.21 
Total time: 237.90 
Avg time per batch: 26.43

Finished Batches: 9 
Time for last batch: 0.19 
Total time: 238.09 
Avg time per batch: 23.81

Finished Batches: 10 
Time for last batch: 46.27

KeyboardInterrupt: 

In [ ]:
learn = Learner(dls, 
                create3Dcnn(n_mid=200, n_out=len(dls.vocab)), 
                loss_func=MixedLoss(norm='euc', alpha=1), 
                splitter=my_splitter, 
                metrics=tl_accuracy)

In [ ]:
torch.cuda.empty_cache()

learn.fine_tune(8)
learn.export(path_charades/'trained_models')